# Exercise 5 - Image classification with Feature Maps (30 Points)

In this exercise you will learn how feature maps can be used improve the accuracy of image classifiers based on softmax regression.

In the event of a persistent problem, do not hesitate to contact the course instructors under
- christoph.staudt@uni-jena.de

### Submission

- Deadline of submission:
        26.05.2021 23:59
- Submission on [moodle page](https://moodle.uni-jena.de/course/view.php?id=28746)

### Installing all required packages

You will need pytorch for this exercise (see more at the end), there are often issues with existing packages and interdependencies. If you use conda you can use the following command to create a completely new environment with all required packages that should work quite well:
`conda create -n torch -c pytorch -c conda-forge scikit-learn numpy matplotlib jupyter pandas pytorch torchvision tqdm`

## Softmax Regression

In this classification task we have more than two classes, therefore we use *softmax regression* as the natural extension of logistic regression (see exercise 2).

Recall, that in softmax regression we have features $x\in\mathbb{R}^n$ which we want to classify into $k$ classes.
The goal is then to find a matrix $\theta\in\mathbb{R}^{n\times k}$, so that

\begin{equation}
p(y=i|x) = \cfrac{\exp\left(x^T\theta^{(i)}\right)}{\sum_{j=1}^{k}\exp\left(x^T\theta^{(j)}\right)}
\end{equation}

where $\theta^{(i)}$ is the i-th column of $\theta$.

## Dataset

For this exercise we use a preprocessed subset of a dataset for [Scene Classification](https://www.kaggle.com/nitishabharathi/scene-classification). All images are 128x128 grayscale images from one of three classes:
- Buildings
- Forest
- Mountains

The goal of this exercise is to train classifiers that can classify images into their respective scenery.

### Task 1 (1 Point)

The dataset is stored as numpy arrays under `X.npy` (images) and `Y.npy` (labels).

Load the dataset and display an image per class using matplotlib. You need to reshape X to the right dimensions again, to show the proper image.

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')

# TODO display images

# No feature map

A first approach for Softmax Regression would be to take the raw pixels as features.

### Task 2 (3 Points)

In scikit-learn, softmax regression is implemented within the [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) class. 

Use scikit learn to fit softmax regression on the images and evaluate the accuracy on train (75%)- and test (25%) data.
What do you observe?

Hint: 
- Use the keyword "multinomial"
- Ignore the max_it warning and that it didn't converge you will see the accuracy on the training set is already pretty high and we cannot expect the accuracy on the test set to get much higher. This is also true for the remaining Tasks

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

# TODO: fit softmax regression

# TODO: get accuracy on train- and test data

assert np.isclose(train_acc,0.98, 0.05)
assert np.isclose(test_acc,0.43, 0.05)

## Normalization

You probably observed, that softmax regression on the raw pixel values does not lead to good results. While the training accuracy is quite high, the test accuracy is very low.

One reason for this is that pixel values are *unnormalized*. This means, different dimensions of the features are in different ranges and have different statistical properties.

When we use a regressor, we heavily rely on the dot product between two vectors as distance measure.

Recall, that for two vectors $x^{(1)}, x^{(2)}\in\mathbb{R}^d$ the dot product is defined as

\begin{align}
(x^{(1)})^Tx^{(2)} = \sum_{i=1}^d x^{(1)}_ix^{(2)}_i
\end{align}

In this distance measure, products over different feature dimensions are added together. If we dont want certain dimensions to dominate others, we have to normalize the features.

### Task 3 (2 Points)

Visualize the mean values and the variance for each feature. What do you observe?

Hint:
- Reshape the values into image dimensions

In [ ]:
# TODO: Visualize mean and variance of pixel features

### Task 4 (2 Points)

Normalize the dataset, so that each feature dimension has mean 0 and standard deviation 1.
Similar to task 4, visualize the mean and variance values for the features of the normalized data.

In [ ]:
# TODO: normalize images

# TODO: visualize mean and variance of normalized pixel features

### Task 5 (1 Point)
Now use logistic regression on the transformed features. Calculate the accuracies on the train- and testset.

In [ ]:
# TODO: use logistic regression on normalized features
assert np.isclose(train_acc,0.999, 0.05)
assert np.isclose(test_acc,0.523, 0.05)

Altough we normalized the features, our accuracy is still quite low.

## Task 6 (2 Points)

Another common off the shelf technique is to perform dimensionality reduction using the [principal component analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Write a function that takes trainings and test data as well as a number of components to take from the PCA. It should fit the PCA (use the class linked above) on the trainings data, then transform the the trainigns and test data, fit the logistic regression on the transformed trainings data and finally return the accuracy on the training and test data.

Run the function for different numbers of components as given in the vairable `components`, print the resulting accuracies.

In [ ]:
def validate_pca(x_train, y_train, x_test, y_test,n_components=3):
    # TODO: Fit x_train for n_components

    # TODO: Transform x_train and x_test

    # TODO: fit regression model on transformed x_train

    # TODO: Calculate accuarcy

components = [3,5,10,30,50]


Using the PCA improved the performance quite a bit, but it is still not very good.

This is because we take pixel values as features, which is extremely inconsistent. 

# Handcrafted features

**Feature maps** attempt to bypass this inconsistency by transforming the pixel vector into a different, more consistent feature vector. 

Especially in the field of image classification, a lot of attempts were made to derive features from images trough a pipeline of preprocessing.

We want to use edges in our images as features.
Edges are visual boundaries of objects. Hence they contain information about objects in images.
In grayscale images, we can interpret edges as sudden changes of pixel intensities in some direction.

## Convolution
In order to detect these changes, we will use linear filters on the images. A linear filter is determined by a kernel matrix $K$, that describes how a new pixel is created by a linear combination of its neighbouring pixels.

As an example, consider the following [kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing)) matrix:

\begin{align}
K=\begin{bmatrix}
\frac{1}{9}&\frac{1}{9}&\frac{1}{9}\\
\frac{1}{9}&\frac{1}{9}&\frac{1}{9}\\
\frac{1}{9}&\frac{1}{9}&\frac{1}{9}
\end{bmatrix}\end{align}

For the calculation of a new pixel, we take the $3\times3$ neighbourhood, i.e. the 8 pixels around the original pixel and the pixel itself, multiply each of these pixels with its corresponding entry in $K$ and sum everything up. The example kernel will simply set a pixel to the average of its neighbouring pixels and is called *average filter*. We consider only kernels with *odd dimensions*.

For the pixels at the border of the image, there is no full neighbourhood and we will assume those missing pixels to be 0. Numpy has the [pad](https://numpy.org/doc/stable/reference/generated/numpy.pad.html) function, that takes an array and `pad_width` and by default adds `pad_width` zeros on each side of the array. Since numpby 1.20 there is another useful function for this task: [sliding_window_view](https://numpy.org/devdocs/reference/generated/numpy.lib.stride_tricks.sliding_window_view.html) takes an array and a window size (the kernel size for us) and returns a view (thus it doesn't copy the data) that has two new dimensions with the same size as the given kernel size. It's first two dimensions are a bit smaller since it always takes a full window and does not use the border pixels. If you combine pad with the sliding window you can create an array that has the correct neighbourhood at i,j which is a matrix with the same size as the kernel:
`sliding_window_view(padded_array,kernel.shape)[i,j] =  neighbourhood[i,j]`.

Finally this can be combined with [einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) to create a fast convolution. For an introduction you can have a look at this [blog post](https://ajcr.net/Basic-guide-to-einsum/). To get the right explicit subscripts string, you need to think about the following questions:
1. How many dimensions / indices does each array have?
2. Which of those should get the same name? Remember that indices can only have the same name if they have the same dimension. By this limitation alone its often clear which indices should be repeated on the left side of the '->'.
3. How many dimensions should the result have and which of the indices should be in it? Remeber you will sum over all indices missing in the result. This will give you the right side of the '->'.

If you can't get einsum to work, please reach out or just use several for loops as indicated in the linked wikipedia article. That will be quite slow however.

### Task 6 (4 Points)
Implement a function, that performs convolution on an image with a given kernel. Consider neighbourhood values that are not within the image as zero (zero padding). Filter the third image from the dataset with the average filter from the example and display the result using matplotlib.

In [ ]:
def convolute(img, kernel):
    # TODO: Implement convolution function
    pass

# TODO: perform average filtering on the third image
img = X[2].reshape((128,128))
kernel = # 3x3 average kernel from above
res = convolute(img,kernel)
assert np.isclose(res[0,0],78.44444444,0.001)
assert np.isclose(res[1,1],175,0.001)
# Plot res

As mentioned before, edges correspond to a sudden change of the pixel intensities. 
We can detect these changes in horizontal and vertical direction with the kernel matrices

\begin{align*}
K_x = \begin{bmatrix}
-1&0&1\\
-2&0&2\\
-1&0&1
\end{bmatrix}\end{align*} 
\begin{align*}K_y = \begin{bmatrix}
-1&-2&-1\\
0&0&0\\
1&2&1
\end{bmatrix}\end{align*}

These kernel matrices are called *Sobel Filters* and serve as an approximation to the gradient of the pixel values.
Let $I_x$ and $I_y$ be the images obtained by filtering our original image with $K_x$ and $K_y$ respectively.

The magnitude of our gradient in both directions can be calculated by 

\begin{align*}
|G| = \sqrt{I_x^2+I_y^2}
\end{align*}


### Task 7 (3 Points)

Implement a function that calculates the gradient image for a given image. Test this function with the third image and visualize it.

In [ ]:
def grad_img(img):
    # TODO: Calculate gradient image (2 Points)
    pass
    
# TODO: calculate + display gradient image
img = X[2].reshape((128,128))
grad = grad_img(img)
# TODO: Plot grad

assert np.isclose(grad[0,0], 745.3026231,0.001)
assert np.isclose(grad[1,1], 21.21320344,0.001)

### Task 8 (2 Points)

Now we want to use this function to create a transformed dataset. Implement a function that takes a dataset of images and transforms it into a dataset of gradient images. Apply this function to the train- and testset.

Note: This might take a while. Have a look at the python package [tqdm](https://tqdm.github.io/) to visualize the conversion process.

In [ ]:
def transform_grad(X):
    # TODO: transform dataset into gradient image vectors
    pass

# TODO: transform train- and testset

### Task 9 (1 Point)
Now use logistic regression on the transformed trainset. What are the accuracies on the train- and testset (Print them!)? Use your function for PCA from above with the different numbers of components on the transformed datasets as well. 

In [ ]:
# TODO: apply softmax regression on transformed train- and testset

# TODO: Apply function for pca with various component sizes again

# Deep Learning Features

In practice, Deep Learning features are the state of the art for most image classification tasks.

Deep Learning features are obtained by processing an input feature with a trained *deep neural network*.
Such a neural network consists of multiple processing steps (*layers*) that are applied sequentially to the input feature.
The resulting features can be used for multiple tasks, such as classification using softmax regression.

We will not cover the topic of deep learning here, we rather view neural nets as a blackbox that works as a feature map.

## Pytorch

We will use a pretrained deep neural network from the deep learning library [Pytorch](https://pytorch.org/).

Install Pytorch and run the imports below.

In [ ]:
# TODO: make sure you can import this
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms

## ResNet18

[ResNet18](https://arxiv.org/pdf/1512.03385.pdf) is a special case of a Deep Learning architecture, that was developed by Microsoft in 2015. It is called ResNet18, because it has 18 layers. 

We want to use ResNet18 as a feature map, that transforms our pixel feature vectors in some other representation. 

<div>
<img src="dl_features.png" width="300"/>
</div>

Pytorch offers the opportunity to use several [pretrained models](https://pytorch.org/vision/0.8/models.html). 
The pretrained ResNet18 was trained to classify images from [ImageNet](http://www.image-net.org/). 
ImageNet is a image classification challenge, where RGB images have to be classified into one of 1000 classes.
Therefore the output of ResNet18 is a vector of size 1000, with the entries showing the confidence in the respective class.

The last layer in ResNet18 corresponds to $\theta$ from the softmax regression. However the pretrained ResNet18 was trained on a different image classification task. Therefore this $\theta$ is of no good for us. We are only interested in the features vectors that are fed into the softmax regression.

An easy way to ignore the final softmax layer and to output the input to this layer is to replace it with a dummy layer, that does nothing but return its input. Below you find an example:

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
model = models.resnet18(pretrained=True)
model.fc = Identity()

As mentioned before, ResNet18 was trained on a different dataset and therefore has special requirements to its inputs.

ResNet18 expects its input to be
- a torch tensor
- of shape $b\times 3\times 224\times 224$, with $b$ as the batch size (number of images that are processed at once)
- normalized in a special way

Pytorch provides all the functionality to preprocess our grayscale input images into the desired form. 

The transformation `preprocess` acts as a function that does all the necessary preprocessing.

In [ ]:
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.cat([x,x,x], axis=0)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: x.unsqueeze(0))
])

### Task 10 (2 Points)

Use the preprocess transformation on an image from the dataset. Preprocess is a function that takes an image (or a batch more on that below) as argument.  
Feed this preprocessed image trough the ResNet18 model.
Print the shapes of the preprocessed features and the ResNet18 features.

Hints:
- look [here](https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py) for examples of the transforms.
- look [here](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html) for an example of a neural network usage in Pytorch (We will only need )

In [ ]:
# TODO: use preprocess on image + print shape

# TODO: feed preprocessed trough model + print shape

### Task 11 (5 Points)

Now transform the train and testset into ResNet18 features. Implement a function, that converts a given dataset into ResNet18 features. Use this function on the train- and testset.

**Important**: Run your transformation within the python [context](https://book.pythontips.com/en/latest/context_managers.html) `with torch.no_grad():` to reduce memory, since we dont need the gradients.

Hints:
- You can feed multiple images trough the model at once, if you [pack them together](https://pytorch.org/docs/stable/generated/torch.cat.html) in a batch

In [ ]:
def transform_dl(X, batchsize=50):
    # TODO: convert images from X into ResNet18 features
    pass

# TODO: transform train- and testset

### Task 12 (2 Points)

Use these features to perform logistic regression. What is the accuracy on the train- and testset now?

In [ ]:
# TODO: apply softmax regression on transformed train- and testset